**Задание 1**

1. Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [1]:
import pandas as pd
from datetime import datetime

In [139]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()


rating_count = ratings.groupby(['userId']).count()
rating_count_100 = rating_count[rating_count['rating'] > 100]

rating_timestamp_agg = ratings.groupby(['userId']).agg(['max','min'])['timestamp'].reset_index()
rating_timestamp_agg['life_time'] = rating_timestamp_agg['max'] - rating_timestamp_agg['min']

life_time = rating_count_100.merge(rating_timestamp_agg, on='userId')[['userId', 'life_time']]
print(f'Cреднее время жизни пользователей {(life_time.mean()["life_time"])/(60*60*24):.1f} дня/дней')


Cреднее время жизни пользователей 463.9 дня/дней


In [94]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')

rat_agg = ratings.groupby('userId').agg({'timestamp' : ['min', 'max', 'count']})
rat_agg['live'] = rat_agg['timestamp']['max'] - rat_agg['timestamp']['min']
(rat_agg[rat_agg['timestamp']['count']  > 100].mean()/(60*60*24))['live']

    463.894762
dtype: float64

**Задание  2**

2. Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:
rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов



In [140]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [141]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [142]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [143]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [186]:
client_base_1 = (
client_base
 .merge(rzd, on='client_id', how='outer') # железнодорожные перевозки
 .merge(auto, on='client_id', how='outer') # автомобильные перевозки
 .merge(air, on='client_id', how='outer') # воздушные перевозки
)

client_base_1.fillna(0, inplace=True)

client_base_2 = client_base_1.copy()

client_base_1.drop(['address'], axis='columns', inplace=True)

client_base_1

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [187]:
client_base_2

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0


**Задание 3**

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.


**1**
- по известным координатам заполняем данные (ширина- lat, долгота - lng)
- в данные добавить время, когда были получены координаты (timestamp)
- связать время покупки(визита) с временем (дельтой времени) timestamp 
- в дальнейшем предлагать пользователю товары, когда он находится по определенным координатам

**2**
- в зависимости от значений координат предлогать товары из списков buying_list, visit_list

**3**
- координаты помогают сопоставить предпочтения и местонахождения (места пребывания) пользователей.
- можно определить: страну, город, курорт, зону отдыха, места командировок и др.

In [202]:
data = pd.DataFrame( 
    {'client_id': [1, 2], 
     'lat' : ['41.40338', '42.73893'], 
     'lng':['2.17403', '3.45343'], 
     'timestamp':[1594580615, 1594583467],
     'buying_list': [[35325],[34]],
     'visit_list': [[12342, 24242, 23422],[]]
    })
data.head()

,client_id,lat,lng,timestamp,buying_list,visit_list
0,1,41.40338,2.17403,1594580615,[35325],"[12342, 24242, 23422]"
1,2,42.73893,3.45343,1594583467,[34],[]
